## Import

In [17]:
%load_ext autoreload
%autoreload 2

from initial_pop import *
from code_classes import *
from Operators.mutations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Population

In [5]:
class Player:
    def __init__(self, name, position, skill, cost):
        self.name = name
        self.position = position
        self.skill = skill
        self.cost = cost

    def __str__(self):
        return f"{self.name} ({self.position}) - Skill: {self.skill}, Cost: {self.cost}M"


In [6]:
class Team:
    def __init__(self, players):
        self.players = players  # players is a list of Player objects
        self.validate_team()

    def validate_team(self):
        positions = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
        for player in self.players:
            if player.position not in positions:
                raise ValueError(f"Invalid player position: {player.position}")
            positions[player.position] += 1

        # Check the required structure
        if positions["GK"] != 1 or positions["DEF"] != 2 or positions["MID"] != 2 or positions["FWD"] != 2:
            raise ValueError("Each team must have 1 GK, 2 DEF, 2 MID, and 2 FWD.")

        # Check if the team exceeds salary cap
        total_salary = sum(player.cost for player in self.players)
        if total_salary > 750:
            raise ValueError(f"Team salary exceeds the cap: {total_salary}M")

    def get_average_skill(self):
        return sum(player.skill for player in self.players) / len(self.players)

    def __str__(self):
        return "\n".join([str(player) for player in self.players])

In [7]:
class League:
    def __init__(self, teams):
        self.teams = teams
        self.validate_league()
                
    def validate_league(self):
        if len(self.teams) != 5:
            raise ValueError("The league must have exactly 5 teams.")

        player_names = set()
        for team in self.teams:
            # Explicitly call team.validate_team()
            team.validate_team()

            for player in team.players:
                if player.name in player_names:
                    raise ValueError(f"Player {player.name} is already in another team.")
                player_names.add(player.name)


    def is_valid(self):
        try:
            self.validate_league()
            return True
        except ValueError:
            return False

    def get_standard_deviation_of_average_skills(self):
        avg_skills = [team.get_average_skill() for team in self.teams]
        return np.std(avg_skills)

    def __str__(self):
        return "\n\n".join([str(team) for team in self.teams])

In [8]:
def calculate_fitness(league):
    if league is None or not league.is_valid():
        return 9999
    return league.get_standard_deviation_of_average_skills()

In [9]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

In [10]:
import random

def create_valid_team_from_pool(player_pool):
    max_attempts = 100
    for _ in range(max_attempts):
        gks = [p for p in player_pool if p.position == "GK"]
        defs = [p for p in player_pool if p.position == "DEF"]
        mids = [p for p in player_pool if p.position == "MID"]
        fwds = [p for p in player_pool if p.position == "FWD"]

        if len(gks) < 1 or len(defs) < 2 or len(mids) < 2 or len(fwds) < 2:
            raise ValueError("Not enough players in the pool to form a valid team.")

        selected_players = random.sample(gks, 1) + \
                           random.sample(defs, 2) + \
                           random.sample(mids, 2) + \
                           random.sample(fwds, 2)

        try:
            team = Team(selected_players)
            return team
        except ValueError:
            continue

    raise ValueError("Failed to create a valid team after many attempts.")

def create_valid_league(all_players, num_teams=5):
    max_attempts = 100
    for _ in range(max_attempts):
        random.shuffle(all_players)
        available_players = all_players.copy()
        used_names = set()
        teams = []

        try:
            for _ in range(num_teams):
                pool = [p for p in available_players if p.name not in used_names]
                team = create_valid_team_from_pool(pool)
                teams.append(team)
                used_names.update(p.name for p in team.players)
            return League(teams)
        except ValueError:
            continue

    raise ValueError("Failed to create a valid league after many attempts.")

def generate_population(players, num_leagues=5):
    population = []
    for _ in range(num_leagues):
        league = create_valid_league(players)
        population.append(league)
    return population


In [11]:
import numpy as np

In [12]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_standard_deviation_of_average_skills():.2f}")


--- League 1 ---
Chris Thompson (GK) Skill: 80 Salary: €80.0M
Owen Parker (DEF) Skill: 88 Salary: €100.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Hunter Cooper (MID) Skill: 83 Salary: €85.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M
Landon Powell (FWD) Skill: 89 Salary: €110.0M

Ryan Mitchell (GK) Skill: 83 Salary: €85.0M
Mason Reed (DEF) Skill: 82 Salary: €75.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Xavier Bryant (FWD) Skill: 90 Salary: €120.0M
Zachary Nelson (FWD) Skill: 86 Salary: €92.0M

Jordan Smith (GK) Skill: 88 Salary: €100.0M
Ethan Howard (DEF) Skill: 80 Salary: €70.0M
Daniel Foster (DEF) Skill: 90 Salary: €110.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Bentley Rivera (MID) Skill: 88 Salary: €100.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M
Adrian Collins (FWD) Skill: 85 Salary: €90.0M

Alex Carter (GK) Skill: 85 Salar

## Test Selection Functions

In [3]:
import random
from collections import Counter
from copy import deepcopy

def test_mutation(population, mutation_fn, mutation_name, mut_prob, trials=100):
    """
    Tests a mutation operator by applying it to each individual in the population
    multiple times and reporting statistics on:
      - valid mutations (new, non-None, changed)
      - invalid mutations (returned None)
      - no-op mutations (returned copy identical to parent)
    """
    overall_stats = Counter()

    for idx, original in enumerate(population):
        orig_str = str(original)
        stats = Counter()
        for _ in range(trials):
            mutated = mutation_fn(original, mut_prob)
            if mutated is None:
                stats['invalid (None)'] += 1
            else:
                # Compare string representations to detect change/no-op
                if str(mutated) == orig_str:
                    stats['no-op (unchanged)'] += 1
                else:
                    stats['valid mutation'] += 1
        # Print per-individual stats
        print(f"{mutation_name} – League #{idx} over {trials} trials:")
        for k, v in stats.items():
            print(f"  {k}: {v}")
        print()
        overall_stats.update(stats)

    # Print overall summary
    print(f"Overall {mutation_name} stats across population ({len(population)} leagues × {trials} trials):")
    total = sum(overall_stats.values())
    for k, v in overall_stats.items():
        pct = v / total * 100
        print(f"  {k}: {v} ({pct:.1f}%)")


### Single-Player Swap

In [13]:
# Test Swap Mutation
test_mutation(
    population,
    single_player_swap_2teams,
    mutation_name="Swap Mutation",
    mut_prob=0.5,
    trials=50
)

Swap Mutation – League #0 over 50 trials:
  no-op (unchanged): 27
  valid mutation: 23

Swap Mutation – League #1 over 50 trials:
  valid mutation: 22
  no-op (unchanged): 27
  invalid (None): 1

Swap Mutation – League #2 over 50 trials:
  valid mutation: 19
  no-op (unchanged): 31

Swap Mutation – League #3 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 26

Swap Mutation – League #4 over 50 trials:
  no-op (unchanged): 28
  valid mutation: 21
  invalid (None): 1

Swap Mutation – League #5 over 50 trials:
  valid mutation: 26
  invalid (None): 1
  no-op (unchanged): 23

Swap Mutation – League #6 over 50 trials:
  valid mutation: 25
  no-op (unchanged): 25

Swap Mutation – League #7 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 25
  invalid (None): 1

Swap Mutation – League #8 over 50 trials:
  no-op (unchanged): 21
  valid mutation: 28
  invalid (None): 1

Swap Mutation – League #9 over 50 trials:
  valid mutation: 22
  no-op (unchanged): 28

Overall Swap Mutatio

### Single-Player All Teams

In [16]:
test_mutation(
    population,
    single_player_shift_all_teams,
    mutation_name="Shift Mutation",
    mut_prob=0.5,
    trials=50
)

Shift Mutation – League #0 over 50 trials:
  no-op (unchanged): 29
  valid mutation: 20
  invalid (None): 1

Shift Mutation – League #1 over 50 trials:
  no-op (unchanged): 17
  valid mutation: 30
  invalid (None): 3

Shift Mutation – League #2 over 50 trials:
  valid mutation: 20
  no-op (unchanged): 28
  invalid (None): 2

Shift Mutation – League #3 over 50 trials:
  valid mutation: 19
  no-op (unchanged): 28
  invalid (None): 3

Shift Mutation – League #4 over 50 trials:
  valid mutation: 29
  no-op (unchanged): 19
  invalid (None): 2

Shift Mutation – League #5 over 50 trials:
  no-op (unchanged): 23
  invalid (None): 5
  valid mutation: 22

Shift Mutation – League #6 over 50 trials:
  valid mutation: 30
  no-op (unchanged): 20

Shift Mutation – League #7 over 50 trials:
  no-op (unchanged): 22
  invalid (None): 4
  valid mutation: 24

Shift Mutation – League #8 over 50 trials:
  valid mutation: 24
  no-op (unchanged): 26

Shift Mutation – League #9 over 50 trials:
  valid mutation

### Full Position Swap

In [19]:
test_mutation(
    population,
    full_position_swap_2teams,
    mutation_name="Full Position Mutation",
    mut_prob=0.5,
    trials=50
)

Full Position Mutation – League #0 over 50 trials:
  no-op (unchanged): 26
  valid mutation: 23
  invalid (None): 1

Full Position Mutation – League #1 over 50 trials:
  no-op (unchanged): 28
  valid mutation: 22

Full Position Mutation – League #2 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 25
  invalid (None): 1

Full Position Mutation – League #3 over 50 trials:
  valid mutation: 14
  invalid (None): 4
  no-op (unchanged): 32

Full Position Mutation – League #4 over 50 trials:
  valid mutation: 24
  no-op (unchanged): 23
  invalid (None): 3

Full Position Mutation – League #5 over 50 trials:
  valid mutation: 25
  no-op (unchanged): 25

Full Position Mutation – League #6 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 26

Full Position Mutation – League #7 over 50 trials:
  valid mutation: 25
  no-op (unchanged): 25

Full Position Mutation – League #8 over 50 trials:
  valid mutation: 24
  no-op (unchanged): 25
  invalid (None): 1

Full Position Mutation – Le